# Cuadrados mínimos lineal

Resolución paso a paso

In [ ]:
import math
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from scipy.stats import chi2

In [ ]:
import danatools

## Datos

In [ ]:
input_file = "cuadrados_minimos_lineal.csv"
data = pd.read_csv(input_file, index_col='i')
data

In [ ]:
fig, ax = plt.subplots()
plt.title("Datos")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.errorbar(data['x'], data['y'], data['dy'],ls='none', marker='o', label="Datos")
# fig.savefig('linear_least_squares_data.svg')

## Estimador de máxima verosimilitud

Covariance matrix of the independent variable $y$

In [ ]:
cova_y = np.diag(data['dy']**2)
cova_y

The fit model is par[0] + par[1] * x

In [ ]:
design_matrix = np.column_stack([data['x']**0, data['x']])
design_matrix

Covariance matrix of the maximum likelihood estimator

In [ ]:
cova_mle = inv( design_matrix.T @ inv(cova_y) @ design_matrix )
cova_mle

In [ ]:
matrix_B = cova_mle @ design_matrix.T @ inv(cova_y)
matrix_B

Maximum likelihood estimator of the parameter $\theta$

In [ ]:
par_est = matrix_B @ data['y']
y0_est = par_est[0]
m_est = par_est[1]
par_est

## Plot del ajuste

In [ ]:
xfit = np.linspace( data['x'].min(), data['x'].max(), num=50)
xfit

In [ ]:
Afit = np.column_stack([xfit**0, xfit])
Afit

In [ ]:
mu_est = Afit @ par_est
mu_est

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.errorbar(data['x'], data['y'], data['dy'],ls='none', marker='o', label="Datos")
ax.plot(xfit, mu_est, ls='--', label="Ajuste")
ax.legend()

## Errores de los parámetros

In [ ]:
dy0 = math.sqrt(cova_mle[0][0])
dm = math.sqrt(cova_mle[1][1])
rho = cova_mle[0][1] / (dy0 *dm)

In [ ]:
print(f'Ordenada al origen:  {y0_est:.3f} ± {dy0:.3f}')
print(f'Pendiente ajustada: {m_est:2.2f} ± {dm:2.2f}')
print(f'Correlación: {rho:2.2f}')

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Intercept")
ax.set_ylabel("Slope")
ax.plot(*par_est, 'o')
elipse_1sigma = danatools.get_ellipse(par_est, cova_mle, nsigma=1)
ax.plot(*elipse_1sigma, ls='--')
elipse_2sigma = danatools.get_ellipse(par_est, cova_mle, nsigma=2)
ax.plot(*elipse_2sigma, ls='--')

## Bondad del ajuste

In [ ]:
residuos = data['y'] - design_matrix @ par_est
J_min_observado = residuos.T @ inv(cova_y) @ residuos
J_min_observado

In [ ]:
ndatos = len(data['x'])
grados_libertad = ndatos - 2
grados_libertad

In [ ]:
pvalor = chi2.sf(J_min_observado, grados_libertad)
print(f"pvalor = {pvalor*100:.2f}%")

## Banda de error

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.errorbar(data['x'], data['y'], data['dy'],ls='none', marker='o', label="Datos")
ax.plot(xfit, mu_est, ls='--', label="Ajuste")
var_mu_est = cova_mle[0][0] + xfit**2*cova_mle[1][1] + 2*xfit*cova_mle[0][1]
sigma_mu_est = np.sqrt(var_mu_est)
ax.fill_between(xfit, mu_est-sigma_mu_est, mu_est+sigma_mu_est, color='tab:orange', alpha=0.2)
ax.legend()
# fig.savefig('linear_least_squares_fit.svg')